# Generere liste over dubletter mellom Oslo/Hamar og Bergen.

In [1]:
from find_fuzzy_duplicates import find_fuzzy_duplicates
from hentOgVaskData import hent_og_vask_data

import polars as pl
pl.Config.set_tbl_rows(-1)

polars.config.Config

In [2]:
'''
1. Leser inn Data exportert i fra Bergens Access database.
   I fra den tabellen som heter `tblInnmeldte`. Dette gjøres ved å benytte funksjonen `hent_og_vask_data()`. Denne funksjonen er også hardkodet med de som har dobbelt medlemskap mellom registerene. Disse blir satt til "venn" i stedet for medlem av Bergens registeret.
'''
df_bergen = hent_og_vask_data()

Could not determine dtype for column 43, falling back to string


In [3]:
#df_bergen.sort(["FORNAVN", "EFTERNAVN"])

# Hamar registeret tilpasses Oslo uttrekk

In [4]:
'''
2. forberedelse for å slå sammen Oslo, Bergen registrene i samme tabell. Dette gjøres for å finne dubletter mellom registrene og egen dubletter. Vi velger å tilpasse Bergen tabellen, slik at det blir like kolonne navn med Oslo registeret. 
'''


df_convTable = df_bergen.select(
    # Columns that are renamed (or kept with the same name)
    pl.lit(0).cast(pl.Int64).alias("Cornerstone-ID"),
    pl.col("MedlemID"),  # Alias is optional when name is the same
    pl.col("Personnummer").alias("Fødselsnummer"),
    pl.col("FORNAVN").alias("Fornavn"),
    pl.lit(None).alias("Mellomnavn"),
    pl.col("EFTERNAVN").alias("Etternavn"),
    pl.col("ADRESSE").alias("Adresse"),
    pl.col("POSTNR").alias("Postnummer"),
    pl.lit(None).cast(pl.String).alias("Gruppens navn 1"),
    pl.col("Kategori").alias("Medlemstype 1"),
    pl.col("Innmeldingsdato").dt.strftime("%d.%m.%Y").alias("Innmeldt dato 1"),
    pl.col("UtmeldtDato").dt.strftime("%d.%m.%Y").alias("Utmeldt dato 1"),
    pl.col("Telefon mob").alias("Telefonnummer 1"),
    pl.col("E-postadresse").alias("E-post"),
    pl.col("RegDato").dt.strftime("%d.%m.%Y").alias("Opprettet"),
)


In [5]:
df_convTable.schema

Schema([('Cornerstone-ID', Int64),
        ('MedlemID', Int64),
        ('Fødselsnummer', String),
        ('Fornavn', String),
        ('Mellomnavn', Null),
        ('Etternavn', String),
        ('Adresse', String),
        ('Postnummer', String),
        ('Gruppens navn 1', String),
        ('Medlemstype 1', String),
        ('Innmeldt dato 1', String),
        ('Utmeldt dato 1', String),
        ('Telefonnummer 1', String),
        ('E-post', String),
        ('Opprettet', String)])

In [6]:
'''
3. Leser inn Oslo/Hamar registeret i en tabell, Vi slår også sammen "Fornavn" og "Mellom" navn, slik at det blir mest mulig likt med Bergen.
'''
# Leser inn en eksport av alle personer fro Cornerstone
df_oslo = (pl.read_excel("oslo_hamar.xlsx")
)

# Slår sammen fornavn og etternavn, slik at det blir likt med Bergens registeret
df_oslo = df_oslo.with_columns(
    pl.when(pl.col("Mellomnavn").is_not_null())  # The condition: IF Mellomnavn is NOT null
    .then(pl.col("Fornavn") + " " + pl.col("Mellomnavn")) # THEN: concatenate the names
    .otherwise(pl.col("Fornavn")) # OTHERWISE: keep the original Fornavn
    .alias("Fornavn") # Crucial: This overwrites the existing "Fornavn" column
)

In [7]:
'''
4. Sjekker om Oslo og Bergen tabellene er like, da er de klare for sammenslåing.
   Slår sammen Oslo og Bergen tabellene til en stor felles tabell
'''

# Sjekker om de to tabellene er like for samenslåing og dublett finning
print(df_oslo.schema)
print(df_convTable.schema)
df_combined = pl.concat([df_oslo, df_convTable])

Schema([('Cornerstone-ID', Int64), ('MedlemID', Int64), ('Fødselsnummer', String), ('Fornavn', String), ('Mellomnavn', String), ('Etternavn', String), ('Adresse', String), ('Postnummer', String), ('Gruppens navn 1', String), ('Medlemstype 1', String), ('Innmeldt dato 1', String), ('Utmeldt dato 1', String), ('Telefonnummer 1', String), ('E-post', String), ('Opprettet', String)])
Schema([('Cornerstone-ID', Int64), ('MedlemID', Int64), ('Fødselsnummer', String), ('Fornavn', String), ('Mellomnavn', Null), ('Etternavn', String), ('Adresse', String), ('Postnummer', String), ('Gruppens navn 1', String), ('Medlemstype 1', String), ('Innmeldt dato 1', String), ('Utmeldt dato 1', String), ('Telefonnummer 1', String), ('E-post', String), ('Opprettet', String)])


# Legg inn dette om du vil ta ut forhåndsdubletter

``` python
df_forhånds_dubletter = (
    pl.read_excel("DubletterOsloBergen.xlsx")
        .filter(pl.col("Menighet") == "BERGEN") #
        .select(pl.col("MedlemID"))
)
df_forhånds_dubletter


df_combined = (
    df_combined.join(
        df_forhånds_dubletter,
        on="MedlemID",
        how="anti"
    )
)
df_combined
```

In [8]:


#df_combined_duplicates.write_excel("dummy.xlsx")

In [ ]:
'''
5. Finner dubletter mellom tabellene ved funksjonen `find_fuzzy_duplicates`
   Tabellen viser grupper av Dubletter 
'''

# Finner dubletter mellom Bergen Oslo/Hamar
df_combined_duplicates = find_fuzzy_duplicates(
    df=df_combined,
    columns=["Fornavn", "Etternavn"],
    columns_to_view=["Mellomnavn","Cornerstone-ID", "MedlemID", "Medlemstype 1" ,"Adresse", "Postnummer", "Innmeldt dato 1","Utmeldt dato 1", "Fødselsnummer"],
    #columns_to_view=None,
    similarity_threshold=90,
    #blocking_column="Postnummer"
)
#print(df_combined_duplicates.select("fuzzy_group","MedlemID", "Fornavn", "Etternavn", "Adresse", "Postnummer","Fødselsnummer"))
df_combined_duplicates

In [19]:
'''
6. Finner Dubletter med flere en 2 forekomster. Dette viser en fin liste over de med mer en 2 forekomster. Tabellen som skrivers ut i 5.2 skal redigeres manuelt til kun å inneholde 2 forekomst dubletter.
'''

# --- Find fuzzy_group values that repeat more than 2 times ---
df_groups_with_more_than_2_members = (
    df_combined_duplicates
    .group_by("fuzzy_group")
    .agg(
        pl.count().alias("group_size") # Count rows in each group
    )
    .filter(
        pl.col("group_size") > 2 # Keep only groups where size > 2
    )
    .select("fuzzy_group", "group_size") # Select just the fuzzy_group column
)

print(df_groups_with_more_than_2_members)


# wiew those who is more than dubeled
df_more_than_2_occurrences = (
    df_groups_with_more_than_2_members
        .join(df_combined_duplicates,
              on="fuzzy_group",
              how="inner"
    )
)
df_more_than_2_occurrences

shape: (9, 2)
┌─────────────┬────────────┐
│ fuzzy_group ┆ group_size │
│ ---         ┆ ---        │
│ i64         ┆ u32        │
╞═════════════╪════════════╡
│ 27          ┆ 4          │
│ 38          ┆ 3          │
│ 41          ┆ 3          │
│ 43          ┆ 3          │
│ 69          ┆ 3          │
│ 72          ┆ 3          │
│ 89          ┆ 3          │
│ 91          ┆ 3          │
│ 94          ┆ 3          │
└─────────────┴────────────┘


/tmp/ipykernel_28440/2168568234.py:8: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("group_size") # Count rows in each group


fuzzy_group,group_size,Fornavn,Etternavn,Mellomnavn,Cornerstone-ID,MedlemID,Medlemstype 1,Adresse,Postnummer,Innmeldt dato 1,Utmeldt dato 1,Fødselsnummer
i64,u32,str,str,str,i64,i64,str,str,str,str,str,str
27,4,"""Gabriel Gaupseth Christiansen""","""Eikaas""",null,2058397,0,"""Medlem Barn""","""Granvegen 54""","""2742""","""01.01.2019""",null,"""05111790568"""
27,4,"""Liv Christiansen""","""Eikaas""",null,2058233,0,"""Medlem""","""Granvegen 54""","""2742""","""01.01.2019""",null,"""31057745811"""
27,4,"""Olivia Viola Gaupseth Christia…","""Eikaas""",null,2058395,0,"""Medlem Barn""","""Granvegen 54""","""2742""","""01.01.2019""",null,"""28041089891"""
27,4,"""Theodor Christiansen""","""Eikaas""",null,2058396,0,"""Medlem Barn""","""Granvegen 54""","""2742""","""01.01.2019""",null,"""23060879923"""
38,3,"""Krister""","""Halck""",null,6040239,0,null,null,null,null,null,null
38,3,"""Krister""","""Halck""",null,4583437,0,null,null,null,null,null,null
38,3,"""Per Krister""","""Halck""",null,2057164,0,null,"""Flesåstunet 2 B""","""0860""",null,null,"""24117037343"""
41,3,"""Lars""","""Hektoen""",null,2055481,0,null,"""Krags vei 9 A""","""0783""",null,null,null
41,3,"""Lars Michael Stockhausen""","""Hektoen""",null,2056516,0,"""Ordinært medlem""","""Krags vei 9""","""0783""","""01.01.2019""",null,"""08098940996"""


In [12]:
#df_combined_duplicates.write_excel("DubletterOsloBergen.xlsx")

In [13]:

df_combined_duplicates_bergen =(
    df_combined_duplicates.join(df_bergen,
                    on="MedlemID",
                    how="inner"
    ).sort("fuzzy_group")
)
df_combined_duplicates_bergen

fuzzy_group,Fornavn,Etternavn,Mellomnavn,Cornerstone-ID,MedlemID,Medlemstype 1,Adresse,Postnummer,Innmeldt dato 1,Utmeldt dato 1,Fødselsnummer,Innmeldingsdato,Er den innmeldte døpt ?,Menighet,Fødested,Dåpsdato,Ant barn under 15 år,Dåpskirke/sted,Fødselsdato,Personnummer,EFTERNAVN,FORNAVN,Stilling,ADRESSE,E-postadresse,POSTNR,Og,SAM_FOR,SAM_EFTER,Utsendelse,Utsendelses kode,Telefon priv,Telefon mob,Sivilstand,Trossamfunn,Gift?,Utsendelsesformat,Ektemakens trossamfunn,Anmerkning,Kategori,Giro,RegDato,SamID,FarID,MorID,Kjønn,Svart?,Faddere,Er mor og far gift?,Prest,Dåpsattest,Søk,UtmeldtDato,DødsDato,Kommune,KID-nr
i64,str,str,str,i64,i64,str,str,str,str,str,str,date,bool,str,str,date,i64,str,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,bool,datetime[ms],i64,i64,i64,str,bool,str,bool,str,bool,str,date,str,str,str
0,"""Janicke""","""Akerø""",null,0,15536,"""M""","""Richard Nordraaks gate 9""","""5018""",null,null,"""07115936284""",null,false,null,null,null,null,null,1959-11-07,"""07115936284""","""Akerø""","""Janicke""",null,"""Richard Nordraaks gate 9""",null,"""5018""",null,null,null,"""UB""","""B""",null,null,null,"""Kristensamfunnet""",false,"""E""",null,null,"""M""",false,2020-03-20 17:16:23,null,null,13033,null,false,null,false,null,false,null,null,null,null,"""10088100739"""
2,"""Gry Veronica""","""Alsos""",null,0,14800,"""M""","""Solåsen 33B""","""5223""","""07.10.2006""",null,"""22077028874""",2006-10-07,false,"""Bergen""",null,null,null,null,1970-07-22,"""22077028874""","""Alsos""","""Gry Veronica""","""Lærer""","""Solåsen 33B""",null,"""5223""",null,null,null,"""UB""","""BS""","""55 10 06 48""",null,null,"""Kristensamfunnet""",false,"""E""","""Kristensamfunnet""",null,"""M""",false,2020-03-20 17:16:57,12729,null,null,"""M""",false,null,false,null,false,null,null,null,null,"""10171100471"""
5,"""Bodil""","""Aukrust""",null,0,12181,"""V""","""Haukedalsbotn 39""","""5113""","""22.12.2000""",null,"""04084728247""",2000-12-22,false,null,null,null,null,null,1947-08-04,"""04084728247""","""Aukrust""","""Bodil""",null,"""Haukedalsbotn 39""",null,"""5113""","""og""","""Erik Øystein""","""Hauglin""","""UB""","""B""","""55 18 28 52""",null,null,null,false,"""P""",null,null,"""V""",true,2022-02-04 16:44:28,null,null,null,"""K""",false,null,false,null,false,null,null,null,"""BERGEN""",null
6,"""Milla Kristine""","""Aukrust""",null,0,11780,"""V""","""Haukedalsbotn 39""","""5113""",null,null,null,null,false,null,null,null,null,null,1982-12-11,null,"""Aukrust""","""Milla Kristine""",null,"""Haukedalsbotn 39""",null,"""5113""",null,null,null,null,null,null,null,null,null,false,null,null,"""Bodil Aukrust, Erik Øystein Ha…","""V""",true,2000-11-11 22:54:18,null,null,12181,"""K""",false,null,false,null,false,null,null,null,"""BERGEN""",null
9,"""Erling""","""Bakken""",null,0,15610,"""V""","""Sanddalsvegen 11 D""","""5221""",null,null,null,null,false,null,null,null,null,null,null,null,"""Bakken""","""Erling""",null,"""Sanddalsvegen 11 D""",null,"""5221""",null,null,null,null,null,null,null,null,null,false,null,null,null,"""V""",false,2019-12-11 10:18:51,null,null,null,null,false,null,false,null,false,null,null,null,null,null
10,"""Jorun""","""Barane""",null,0,15537,"""M""","""Nedstagata 8""","""5745""",null,null,"""22086139603""",null,false,null,null,null,null,null,1961-08-22,"""22086139603""","""Barane""","""Jorun""",null,"""Nedstagata 8""",null,"""5745""","""og""","""Morten""","""Clemetsen""",null,null,null,null,null,"""Kristensamfunnet""",false,null,null,null,"""M""",false,2014-11-11 23:49:37,14463,null,null,null,false,null,false,null,false,null,null,null,null,null
11,"""John L.""","""Baum""",null,0,14695,"""P""","""Munkerudveien 81E""","""1165""",null,null,null,null,false,null,null,null,null,null,null,null,"""Baum""","""John L.""",null,"""Munkerudveien 81E""",null,"""1165""",null,null,null,null,null,null,null,null,null,false,null,null,null,"""P""",false,2014-10-17 12:29:28,null,null,null,null

In [14]:
'''
7. Skriver ut listen over dubletter til fil. Denne filen skal behandles manuelt. Kollonner skal redigeres. Bruk sunn fornuft og velg data fra den dubletten som er mest mulig up to date. Det er som regel de som har post utsending, eller er medlemmer.
	1. Slett der det er mer en 2 forekomster
	2. Rette på navn, slik at det blir likt mellom dubletter, også mellomnavn
	3. Dersom det er fødselsnummer på en av dublettene, så kopieres dette over på den andre
	4. Skriv inn medlemsnummer og Cornerstone ID på begge forekomster, der det mangler.
	5. Denne listen skal brukes ved konverteringen.
'''
#df_combined_duplicates.write_excel("DubletterOsloBergen.xlsx")